In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Allen - OephysConverter
This tutorial follows the step-by-step guide for a [NWB Converter](https://github.com/catalystneuro/nwb-conversion-tools/blob/master/documentation/conversion_tools_structure.md#step-by-step-operations)

In [ ]:
from allen_oephys_to_nwb import AllenOephysNWBConverter
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
from pathlib import Path
import pprint

## Step 1 - Converter.get_source_schema()

In [ ]:
# Get source_schema
source_schema = AllenOephysNWBConverter.get_source_schema()

print('Data Interfaces source data:')
for k, v in source_schema['properties'].items():
    print(f"{k}: {list(v['properties'].keys())}")

## Step 2 - Get user-input source_data that complies to the returned full source_schema

In [ ]:
# Source data
base_name = 'Emx1-s_highzoom/102086'

path_ecephys_raw = Path.cwd() / 'Emx1-s_highzoom/102086.h5'
path_ecephys_processed = Path.cwd() / 'Emx1-s_highzoom/102086_processed.h5'
path_subjects_info = Path.cwd() / 'Emx1-s_highzoom/subjects_info.json'
path_tiff_red_channel = Path.cwd() / 'Emx1-s_highzoom/102086_2.tif'
path_tiff_green_channel = Path.cwd() / 'Emx1-s_highzoom/102086_1.tif'
path_ophys_processed = Path.cwd() / 'Emx1-s_highzoom/102086_processed.h5'

source_data = dict(
    AllenEcephysInterface=dict(
        path_ecephys_raw=str(path_ecephys_raw),
        path_ecephys_processed=str(path_ecephys_processed),
        path_subjects_info=str(path_subjects_info)
    ),
    AllenOphysInterface=dict(
        path_tiff_red_channel=str(path_tiff_red_channel),
        path_tiff_green_channel=str(path_tiff_green_channel), 
        path_ophys_processed=str(path_ophys_processed),
        path_subjects_info=str(path_subjects_info)
    )
)
pprint.pprint(source_data, width=120)

## Step 3 - Instantiate Converter

In [ ]:
# Initialize converter
converter = AllenOephysNWBConverter(source_data)

print('Data interfaces for this converter:')
pprint.pprint(converter.data_interface_objects, width=120)

## Step 4 - Converter.get_metadata_schema()

In [ ]:
# Get metadata_schema
metadata_schema = converter.get_metadata_schema()

print('Metadata schema items:')
for k, v in metadata_schema['properties'].items():
    print(f"- {k}")
    if k not in ['NWBFile', 'Subject']:
        for kk in v['properties'].keys():
            print(f"---- {kk}")

## Step 5 - Automatically fetches available metadata with Converter.get_metadata()

In [ ]:
# Get metadata from source data
metadata = converter.get_metadata()
pprint.pprint(metadata, width=120)

## Step 6 - Get user-input metadata

In [ ]:
metadata['NWBFile']['session_description'] = 'example conversion'
pprint.pprint(metadata, width=120)

## Step 7 - Converter.get_conversion_options_schema()

In [ ]:
# Not used for this converter
conversion_options_schema = converter.get_conversion_options_schema()
conversion_options_schema

## Step 8 - Get user-input conversion options
Provide dictionary with conversion options according to Converter schema

In [ ]:
conversion_options = dict(
    AllenEcephysInterface=dict(
        add_ecephys_raw=True,
        add_ecephys_processed=True,
        add_ecephys_spiking=True
    ),
    AllenOphysInterface=dict(
        add_ophys_processed=True,
        add_ophys_raw=True,
        link_ophys_raw=False,
    )
)
pprint.pprint(conversion_options, width=120)

## Step 9 - Run conversion user filled metadata and conversion_options

In [ ]:
output_file = 'output.nwb'
save_to_file = True

if save_to_file:
    converter.run_conversion(
        metadata=metadata, 
        nwbfile_path=output_file, 
        save_to_file=True,
        conversion_options=conversion_options
    )
else:
    nwbfile = converter.run_conversion(
        metadata=metadata, 
        nwbfile_path=output_file, 
        save_to_file=False,
        conversion_options=conversion_options
    )

## Final 1 - Check NWB file

In [ ]:
# load file
with NWBHDF5IO(output_file, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

## Final 2 - Check NWB file with widgets

In [ ]:
io = NWBHDF5IO(output_file, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)